In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2021 data into respective dataframes
df1 = pd.read_csv('../Data/2021/202101-divvy-tripdata.csv')
df2 = pd.read_csv('../Data/2021/202102-divvy-tripdata.csv')
df3 = pd.read_csv('../Data/2021/202103-divvy-tripdata.csv')
df4 = pd.read_csv('../Data/2021/202104-divvy-tripdata.csv')
df5 = pd.read_csv('../Data/2021/202105-divvy-tripdata.csv')
df6 = pd.read_csv('../Data/2021/202106-divvy-tripdata.csv')
df7 = pd.read_csv('../Data/2021/202107-divvy-tripdata.csv')
df8 = pd.read_csv('../Data/2021/202108-divvy-tripdata.csv')
df9 = pd.read_csv('../Data/2021/202109-divvy-tripdata.csv')
df10 = pd.read_csv('../Data/2021/202110-divvy-tripdata.csv')
df11 = pd.read_csv('../Data/2021/202111-divvy-tripdata.csv')
df12 = pd.read_csv('../Data/2021/202112-divvy-tripdata.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual


In [4]:
dfbike.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

### Saving all the station names in a list "allSt21"

#### Comparing with 2020 data and only adding the stations which are new into a list "added21"
#### Also, checking for the stations that have been in 2020 but not in this year's data and adding it to "removed20"

In [5]:
allSt21 = list(dfbike['start_station_name'].unique())
len(allSt21)

848

In [6]:
allSt20 = ['Western Ave & Leland Ave',
 'Clark St & Montrose Ave',
 'Broadway & Belmont Ave',
 'Clark St & Randolph St',
 'Clinton St & Lake St',
 'Wells St & Hubbard St',
 'Desplaines St & Randolph St',
 'Clark St & Lake St',
 'Wilton Ave & Belmont Ave',
 'LaSalle St & Illinois St',
 'Clinton St & Washington Blvd',
 'California Ave & Milwaukee Ave',
 'Franklin St & Jackson Blvd',
 'Wilton Ave & Diversey Pkwy',
 'Lincoln Ave & Diversey Pkwy',
 'Kingsbury St & Kinzie St',
 'Lake Shore Dr & Monroe St',
 'Canal St & Adams St',
 'Indiana Ave & Roosevelt Rd',
 'Broadway & Sheridan Rd',
 'Desplaines St & Jackson Blvd',
 'Western Ave & Division St',
 'Dearborn St & Erie St',
 'Burling St (Halsted) & Diversey Pkwy (Temp)',
 'Clark St & Newport St',
 'Logan Blvd & Elston Ave',
 'Milwaukee Ave & Wabansia Ave',
 'Ada St & Washington Blvd',
 'Southport Ave & Clybourn Ave',
 'Clybourn Ave & Division St',
 'Larrabee St & Kingsbury St',
 'Milwaukee Ave & Grand Ave',
 'Sheridan Rd & Irving Park Rd',
 'Desplaines St & Kinzie St',
 'Elizabeth (May) St & Fulton St',
 'Carpenter St & Huron St',
 'Lincoln Ave & Roscoe St',
 'Damen Ave & Grand Ave',
 'Michigan Ave & 14th St',
 'California Ave & Montrose Ave',
 'Sheridan Rd & Buena Ave',
 'Green St & Randolph St',
 'Daley Center Plaza',
 'Canal St & Madison St',
 'Franklin St & Monroe St',
 'Columbus Dr & Randolph St',
 'Wabash Ave & Wacker Pl',
 'Wabash Ave & Grand Ave',
 'Ogden Ave & Race Ave',
 'Streeter Dr & Grand Ave',
 'Ashland Ave & Division St',
 'State St & 19th St',
 'Canal St & Taylor St',
 'Michigan Ave & Washington St',
 'Field Blvd & South Water St',
 'Wabash Ave & 16th St',
 'Clark St & Wellington Ave',
 'Kedzie Ave & Milwaukee Ave',
 'Kosciuszko Park',
 'LaSalle St & Washington St',
 'Clark St & Grace St',
 'Ravenswood Ave & Lawrence Ave',
 'Lincoln Ave & Belmont Ave',
 'Larrabee St & Division St',
 'Sheffield Ave & Wrightwood Ave',
 'Halsted St & Wrightwood Ave',
 'Broadway & Barry Ave',
 'Larrabee St & Oak St',
 'Sedgwick St & Huron St',
 'Ellis Ave & 55th St',
 'Harper Ave & 59th St',
 'Lake Park Ave & 35th St',
 'Michigan Ave & Lake St',
 'Clinton St & Jackson Blvd',
 'Morgan St & Polk St',
 'Ogden Ave & Roosevelt Rd',
 'Western Ave & Fillmore St (*)',
 'Ashland Ave & 13th St',
 'Fairbanks St & Superior St',
 'Clarendon Ave & Leland Ave',
 'Morgan St & Lake St',
 'Sangamon St & Washington Blvd',
 'Calumet Ave & 35th St',
 'MLK Jr Dr & Pershing Rd',
 'Indiana Ave & 40th St',
 'Stave St & Armitage Ave',
 'Wood St & Chicago Ave (*)',
 'Southport Ave & Roscoe St',
 'Southport Ave & Wellington Ave',
 'Chicago Ave & Washington St',
 'University Library (NU)',
 'Honore St & Division St',
 'Western Ave & Walton St',
 'Southport Ave & Irving Park Rd',
 'McClurg Ct & Illinois St',
 'Wabash Ave & Adams St',
 'Loomis St & Lexington St',
 'Clinton St & Tilden St',
 'Clark St & Schiller St',
 'Clark St & Wrightwood Ave',
 'LaSalle Dr & Huron St',
 'Wells St & Huron St',
 'Clark St & Drummond Pl',
 'LaSalle St & Jackson Blvd',
 'McClurg Ct & Erie St',
 'Marshfield Ave & Cortland St',
 'Clinton St & Madison St',
 'Western Ave & Monroe St',
 'Michigan Ave & Oak St',
 'Lincoln Ave & Sunnyside Ave',
 'State St & Randolph St',
 'Clark St & Touhy Ave',
 'Eastlake Ter & Rogers Ave',
 'Pine Grove Ave & Waveland Ave',
 'Ellis Ave & 60th St',
 'Lincoln Ave & Belle Plaine Ave',
 'Southport Ave & Waveland Ave',
 'Wells St & Elm St',
 'Sedgwick St & North Ave',
 'Racine Ave & Congress Pkwy',
 'May St & Taylor St',
 'Bissell St & Armitage Ave',
 'Clark St & Elm St',
 'Spaulding Ave & Armitage Ave',
 'Dearborn St & Monroe St',
 'Central Park Ave & Elbridge Ave',
 'Kedzie Ave & Palmer Ct',
 'Paulina St & Howard St',
 'Sheffield Ave & Fullerton Ave',
 'Halsted St & Clybourn Ave',
 'Ravenswood Ave & Irving Park Rd',
 'Lake Shore Dr & Wellington Ave',
 'Franklin St & Lake St',
 'Adler Planetarium',
 'Clark St & Bryn Mawr Ave',
 'State St & Kinzie St',
 'Wells St & Walton St',
 'Orleans St & Chestnut St (NEXT Apts)',
 'Sheffield Ave & Kingsbury St',
 'California Ave & Fletcher St',
 'Halsted St & 18th St',
 'Eckhart Park',
 'Blue Island Ave & 18th St',
 'State St & Van Buren St',
 'Michigan Ave & Madison St',
 'Michigan Ave & 8th St',
 'Wabash Ave & 9th St',
 'Clinton St & Roosevelt Rd',
 'Stetson Ave & South Water St',
 'Orleans St & Hubbard St',
 'Orleans St & Merchandise Mart Plaza',
 'Clark St & Winnemac Ave',
 'Blackstone Ave & Hyde Park Blvd',
 'Shore Dr & 55th St',
 'Stockton Dr & Wrightwood Ave',
 'Lakeview Ave & Fullerton Pkwy',
 'Calumet Ave & 21st St',
 'Lincoln Park Conservatory',
 'Western Ave & Winnebago Ave',
 'Albany Ave & Bloomingdale Ave',
 'Walsh Park',
 'Dearborn Pkwy & Delaware Pl',
 'Fairbanks Ct & Grand Ave',
 'Milwaukee Ave & Rockwell St',
 'Franklin St & Illinois St',
 'Wacker Dr & Washington St',
 'Kingsbury St & Erie St',
 'Racine Ave & Wrightwood Ave',
 'Dearborn St & Van Buren St',
 'Southport Ave & Wrightwood Ave',
 '900 W Harrison St',
 'Canal St & Monroe St',
 'Pine Grove Ave & Irving Park Rd',
 'Broadway & Cornelia Ave',
 'Loomis St & Archer Ave',
 'Clarendon Ave & Gordon Ter',
 'California Ave & Francis Pl (Temp)',
 'Sheffield Ave & Willow St',
 'Clark St & Armitage Ave',
 'Millennium Park',
 'Mies van der Rohe Way & Chicago Ave',
 'Larrabee St & Menomonee St',
 'Larrabee St & North Ave',
 'Clark St & North Ave',
 'Racine Ave & Washington Blvd (*)',
 'Sedgwick St & Webster Ave',
 'Lincoln Ave & Waveland Ave',
 'Field Museum',
 'Wolcott (Ravenswood) Ave & Montrose Ave',
 'Humboldt Blvd & Armitage Ave',
 'Damen Ave & Leland Ave',
 'State St & Pearson St',
 'Broadway & Wilson Ave',
 'Paulina St & Flournoy St',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Winchester (Ravenswood) Ave & Balmoral Ave',
 'Greenwood Ave & 47th St',
 'Lake Shore Dr & North Blvd',
 'Broadway & Ridge Ave',
 'Hermitage Ave & Polk St',
 'Halsted St & Roscoe St',
 'Clark St & Chicago Ave',
 'Wood St & Milwaukee Ave',
 'Damen Ave & Cortland St',
 'Michigan Ave & 18th St',
 'Halsted St & Polk St',
 'Wells St & Polk St',
 'Campbell Ave & Montrose Ave',
 'Drake Ave & Montrose Ave',
 'Franklin St & Adams St (Temp)',
 'Wabash Ave & Roosevelt Rd',
 'Shedd Aquarium',
 'Rush St & Cedar St',
 'Racine Ave & Fullerton Ave',
 'Francisco Ave & Foster Ave',
 'Leavitt St & Armitage Ave',
 'Glenwood Ave & Morse Ave',
 'Oakley Ave & Irving Park Rd',
 'Monticello Ave & Irving Park Rd',
 'Greenview Ave & Fullerton Ave',
 'Ritchie Ct & Banks St',
 'Lake Shore Dr & Diversey Pkwy',
 'Damen Ave & Madison St',
 'Halsted St & Willow St',
 'Halsted St & Dickens Ave',
 'Rush St & Hubbard St',
 'Green St & Madison St',
 'Larrabee St & Webster Ave',
 'Wood St & Hubbard St',
 'Halsted St & Roosevelt Rd',
 'Wood St & Augusta Blvd',
 'State St & Harrison St',
 'Racine Ave & 13th St',
 'Canal St & Harrison St',
 'McCormick Place',
 'St. Clair St & Erie St',
 'Ashland Ave & Wellington Ave',
 'Wells St & Concord Ln',
 'Ashland Ave & Grand Ave',
 'Benson Ave & Church St',
 'Ellis Ave & 53rd St',
 'Ellis Ave & 58th St',
 'Kimbark Ave & 53rd St',
 'Hoyne Ave & Balmoral Ave',
 'University Ave & 57th St',
 'Dodge Ave & Church St',
 'Christiana Ave & Lawrence Ave',
 'Kedzie Ave & Leland Ave',
 'Glenwood Ave & Touhy Ave',
 'Southport Ave & Clark St',
 'Clark St & Lincoln Ave',
 'Campbell Ave & North Ave',
 'Oakley Ave & Touhy Ave',
 'Morgan Ave & 14th Pl',
 'Wells St & Evergreen Ave',
 'Aberdeen St & Monroe St',
 'Ogden Ave & Chicago Ave',
 'Ashland Ave & Wrightwood Ave',
 'Broadway & Argyle St',
 'Bernard St & Elston Ave',
 'Ashland Ave & Chicago Ave',
 'LaSalle St & Adams St',
 'Dearborn St & Adams St',
 'Aberdeen St & Jackson Blvd',
 'Franklin St & Chicago Ave',
 'Ashland Ave & Grace St',
 'Racine Ave & Randolph St',
 'Lakefront Trail & Bryn Mawr Ave',
 'Sheffield Ave & Wellington Ave',
 'State St & 79th St',
 'Ashland Ave & Lake St',
 'Peoria St & Jackson Blvd',
 'Rush St & Superior St',
 'Orleans St & Elm St',
 'Ashland Ave & Belle Plaine Ave',
 'Clifton Ave & Armitage Ave',
 'Ashland Ave & Augusta Blvd',
 'Marine Dr & Ainslie St',
 'Broadway & Thorndale Ave',
 'MLK Jr Dr & 29th St',
 'Indiana Ave & 26th St',
 'Wabash Ave & Cermak Rd',
 'Damen Ave & Pierce Ave',
 'Throop (Loomis) St & Taylor St',
 'Cornell Ave & Hyde Park Blvd',
 'Michigan Ave & Pearson St',
 'Central St & Girard Ave',
 'Sheridan Rd & Lawrence Ave',
 'Lake Park Ave & 53rd St',
 'Broadway & Berwyn Ave',
 'State St & 33rd St',
 'Drake Ave & Addison St',
 'Bosworth Ave & Howard St',
 'Noble St & Milwaukee Ave',
 'Western Ave & 21st St',
 'Wentworth Ave & Cermak Rd (Temp)',
 'California Ave & Division St',
 'Lincoln Ave & Fullerton Ave',
 'Lake Shore Dr & Ohio St',
 'Clinton St & 18th St',
 'Damen Ave & Clybourn Ave',
 'Damen Ave & Thomas St (Augusta Blvd)',
 'Sacramento Blvd & Franklin Blvd',
 'Richmond St & Diversey Ave',
 'Damen Ave & Chicago Ave',
 'Financial Pl & Ida B Wells Dr',
 'Clark St & Ida B Wells Dr',
 'Michigan Ave & Jackson Blvd',
 'Michigan Ave & Ida B Wells Dr',
 'Aberdeen St & Randolph St',
 'California Ave & Cortez St',
 'Winthrop Ave & Lawrence Ave',
 'Larrabee St & Armitage Ave',
 'Clinton St & Polk St',
 'Avondale Ave & Irving Park Rd',
 'Wentworth Ave & 24th St (Temp)',
 'Wolcott Ave & Polk St',
 'Damen Ave & Division St',
 'Wood St & Taylor St',
 'Jefferson St & Monroe St',
 'Paulina St & 18th St',
 'Clark St & Jarvis Ave',
 'Southport Ave & Belmont Ave',
 'Normal Ave & Archer Ave',
 'Warren Park East',
 'Halsted St & North Branch St',
 'Winchester Ave & Elston Ave',
 'Clark St & Leland Ave',
 'Theater on the Lake',
 'Damen Ave & Charleston St',
 'Broadway & Waveland Ave',
 'Artesian Ave & Hubbard St',
 'Canal St & Jackson Blvd',
 'California Ave & Altgeld St',
 'Delano Ct & Roosevelt Rd',
 'Western Ave & Roscoe St',
 'Dayton St & North Ave',
 'Damen Ave & Wellington Ave',
 'Lake Shore Dr & Belmont Ave',
 'Dusable Harbor',
 'Clark St & Berwyn Ave',
 'California Ave & Byron St',
 'Mies van der Rohe Way & Chestnut St',
 'Leavitt St & North Ave',
 'Greenview Ave & Diversey Pkwy',
 'Calumet Ave & 18th St',
 'Warren Park West',
 'Ravenswood Ave & Berteau Ave',
 'Emerald Ave & 28th St',
 'Paulina St & Montrose Ave',
 'Prairie Ave & Garfield Blvd',
 'Central St Metra',
 'Campbell Ave & Fullerton Ave',
 'Seeley Ave & Roscoe St',
 'Rockwell St & Eastwood Ave',
 'Broadway & Granville Ave',
 'Pulaski Rd & Eddy St (Temp)',
 'Halsted St & Archer Ave',
 'Woodlawn Ave & 55th St',
 'Federal St & Polk St',
 'Elston Ave & Wabansia Ave',
 'Damen Ave & Sunnyside Ave',
 'Ashland Ave & 50th St',
 'Racine Ave & Belmont Ave',
 'Cannon Dr & Fullerton Ave',
 'Sedgwick St & Schiller St',
 'Chicago Ave & Sheridan Rd',
 'Lake Park Ave & 56th St',
 'Emerald Ave & 31st St',
 'Halsted St & 21st St',
 'Indiana Ave & 31st St',
 'Lincoln Ave & Winona St',
 'State St & 35th St',
 'Clark St & Schreiber Ave',
 'Burnham Harbor',
 'Loomis St & Jackson Blvd',
 'Montrose Harbor',
 'Damen Ave & Cullerton St',
 'Western Ave & Congress Pkwy',
 'Washtenaw Ave & Ogden Ave',
 'Lakefront Trail & Wilson Ave',
 'Elmwood Ave & Austin St',
 'Central Park Ave & North Ave',
 'Chicago Ave & Dempster St',
 'Sheffield Ave & Waveland Ave',
 'Western Ave & Granville Ave',
 'Clarendon Ave & Junior Ter',
 'Sheridan Rd & Loyola Ave',
 'Shields Ave & 28th Pl',
 'Smith Park (*)',
 'Morgan St & 18th St',
 'Damen Ave & Foster Ave',
 'Rainbow Beach',
 'Calumet Ave & 33rd St',
 'Damen Ave & Melrose Ave',
 'Lincoln Ave & Addison St',
 'Racine Ave & 18th St',
 'Sheridan Rd & Noyes St (NU)',
 'Clark St & Lunt Ave',
 'Malcolm X College',
 'Ogden Ave & Congress Pkwy',
 'Fort Dearborn Dr & 31st St',
 'Western Ave & Howard St',
 'Halsted St & Maxwell St',
 'Damen Ave & Walnut (Lake) St (*)',
 'Clark St & Elmdale Ave',
 'Sheridan Rd & Greenleaf Ave',
 'Ashland Ave & McDowell Ave',
 'Wolcott Ave & Fargo Ave',
 'Paulina Ave & North Ave',
 'Claremont Ave & Hirsch St',
 'Ashland Ave & Blackhawk St',
 'Drake Ave & Fullerton Ave',
 'Clark St & 9th St (AMLI)',
 'Greenview Ave & Jarvis Ave',
 'Lake Park Ave & 47th St',
 'Morgan St & 31st St',
 'State St & 29th St',
 'Manor Ave & Leland Ave',
 'Sheridan Rd & Montrose Ave',
 'Leavitt St & Chicago Ave',
 'Kedzie Ave & Harrison St',
 'Vernon Ave & 75th St',
 'Greenwood Ave & 79th St',
 'May St & Cullerton St',
 'California Ave & Lake St',
 'Sheffield Ave & Webster Ave',
 'Ashland Ave & Pershing Rd',
 'Leavitt St & Lawrence Ave',
 'Dorchester Ave & 49th St',
 'Leavitt St & Division St (*)',
 'Damen Ave & Coulter St',
 'Kedzie Ave & Foster Ave',
 'Troy St & Elston Ave',
 'Cottage Grove Ave & Oakwood Blvd',
 'Halsted St & 37th St',
 'Leavitt St & Archer Ave',
 'California Ave & North Ave',
 'Clark St & Columbia Ave',
 'Wells St & 19th St',
 'Talman Ave & Addison St',
 'Albany Ave & Montrose Ave',
 'Buckingham Fountain',
 'Wentworth Ave & 35th St',
 'Racine Ave & 35th St',
 'Racine Ave & 15th St',
 'Wallace St & 35th St',
 'Washtenaw Ave & Lawrence Ave',
 'St. Louis Ave & Balmoral Ave',
 'Kimball Ave & Belmont Ave',
 'Princeton Ave & 47th St',
 'Laflin St & Cullerton St',
 'Spaulding Ave & Division St',
 'Stony Island Ave & 71st St',
 'Woodlawn Ave & Lake Park Ave',
 'Cherry Ave & Blackhawk St',
 'South Shore Dr & 67th St',
 'Austin Blvd & Madison St',
 'Leavitt St & Belmont Ave (*)',
 'Rhodes Ave & 32nd St',
 'Kedzie Ave & Lake St',
 'Ridge Blvd & Touhy Ave',
 'Central Park Ave & Bloomingdale Ave',
 'Kilbourn Ave & Milwaukee Ave',
 'Conservatory Dr & Lake St',
 'Western Ave & 24th St',
 'Keystone Ave & Montrose Ave',
 'Elizabeth St & 47th St',
 'California Ave & 23rd Pl',
 'Damen Ave & Pershing Rd',
 'Shields Ave & 31st St',
 'Cottage Grove Ave & 51st St',
 'Western Blvd & 48th Pl',
 'Museum of Science and Industry',
 'Wentworth Ave & 33rd St',
 'Troy St & North Ave',
 'Halsted St & 35th St',
 'Western Ave & Lunt Ave',
 'DuSable Museum',
 'Dodge Ave & Mulford St',
 'Leavitt St & Addison St',
 'South Shore Dr & 71st St',
 'Evanston Civic Center',
 'Lincolnwood Dr & Central St',
 'Dorchester Ave & 63rd St',
 'Stony Island Ave & 67th St',
 'Pulaski Rd & Lake St',
 'Avers Ave & Belmont Ave',
 'Sheridan Rd & Columbia Ave',
 'Sawyer Ave & Irving Park Rd',
 'Calumet Ave & 51st St',
 'Yates Blvd & 75th St',
 '63rd St Beach',
 'MLK Jr Dr & 47th St',
 'Ridge Blvd & Howard St',
 'Racine Ave & Garfield Blvd',
 'Cottage Grove Ave & 47th St',
 'Prairie Ave & 43rd St',
 'Archer (Damen) Ave & 37th St',
 'Cottage Grove Ave & 78th St',
 'Ashland Ave & Archer Ave',
 'Morgan St & Pershing Rd',
 'Princeton Ave & Garfield Blvd',
 'Cottage Grove Ave & 43rd St',
 'Calumet Ave & 71st St',
 'Jeffery Blvd & 71st St',
 'Evans Ave & 75th St',
 'Vernon Ave & 79th St',
 'Cottage Grove Ave & 71st St',
 'Kedzie Ave & 21st St',
 '2112 W Peterson Ave',
 'California Ave & 21st St',
 'Ashland Ave & Garfield Blvd',
 'Stony Island Ave & 64th St',
 'South Chicago Ave & 83rd St',
 'Valli Produce - Evanston Plaza',
 'Halsted St & 56th St',
 'Austin Blvd & Lake St',
 'Kedzie Ave & Chicago Ave',
 'California Ave & 26th St',
 'Kilbourn Ave & Irving Park Rd',
 'Central Ave & Madison St',
 'Millard Ave & 26th St',
 'Wood St & 35th St',
 'Kedzie Ave & 24th St',
 'Wentworth Ave & 63rd St',
 'Kildare Ave & Montrose Ave',
 'Keystone Ave & Fullerton Ave',
 'Cottage Grove Ave & 67th St',
 'Laramie Ave & Gladys Ave',
 'Knox Ave & Montrose Ave',
 'Stony Island Ave & 82nd St',
 'Damen Ave & 51st St',
 'Cottage Grove Ave & 63rd St',
 'Milwaukee Ave & Cuyler Ave',
 'Budlong Woods Library',
 'Laramie Ave & Madison St',
 'Central Park Ave & 24th St',
 'Central Park Ave & Ogden Ave',
 'Marshfield Ave & 44th St',
 'Central Ave & Chicago Ave',
 'Latrobe Ave & Chicago Ave',
 'Stony Island Ave & 75th St',
 'South Shore Dr & 74th St',
 'State St & Pershing Rd',
 'Stony Island Ave & South Chicago Ave',
 'MLK Jr Dr & 56th St',
 'Eggleston Ave & 69th St (*)',
 'Halsted St & 69th St',
 'Albany Ave & 26th St',
 'Maplewood Ave & Peterson Ave',
 'Halsted St & 63rd St',
 'Cicero Ave & Flournoy St',
 'Hoyne Ave & 47th St',
 'Kedzie Ave & Bryn Mawr Ave',
 'Central Park Blvd & 5th Ave',
 'Jeffery Blvd & 67th St',
 'Fairfield Ave & Roosevelt Rd',
 'Ashland Ave & 63rd St',
 'Kostner Ave & Lake St',
 'MTL-ECO5.1-01',
 'State St & 76th St',
 'Cicero Ave & Quincy St',
 'MLK Jr Dr & 63rd St',
 'Throop St & 52nd St',
 'HUBBARD ST BIKE CHECKING (LBS-WH-TEST)',
 'Pulaski Rd & Congress Pkwy',
 'Cornell Dr & Hayes Dr',
 'Bennett Ave & 79th St',
 'Central Ave & Lake St',
 'Austin Blvd & Chicago Ave',
 'Central Ave & Harrison St',
 'Elizabeth St & 59th St',
 'Kedzie Ave & Roosevelt Rd',
 'Michigan Ave & 71st St',
 'Western Ave & 28th St',
 'Union Ave & Root St',
 'Cottage Grove Ave & 83rd St',
 'Phillips Ave & 83rd St',
 'Stewart Ave & 63rd St (*)',
 'Wabash Ave & 87th St',
 'MLK Jr Dr & 83rd St',
 'Wabash Ave & 83rd St',
 'Ellis Ave & 83rd St',
 'May St & 69th St',
 'Halsted St & 47th Pl',
 'Exchange Ave & 79th St',
 'Commercial Ave & 83rd St',
 'HQ QR',
 'Marshfield Ave & 59th St',
 'Laramie Ave & Kinzie St',
 'Halsted St & 51st St',
 'Woodlawn Ave & 75th St',
 'State St & 54th St',
 'Phillips Ave & 79th St',
 'Shields Ave & 43rd St',
 'Eberhart Ave & 61st St',
 'Perry Ave & 69th St',
 'Jeffery Blvd & 76th St',
 'Karlov Ave & Madison St',
 'Carpenter St & 63rd St',
 'Seeley Ave & Garfield Blvd',
 'Kenton Ave & Madison St',
 'Kostner Ave & Adams St',
 'Cicero Ave & Lake St',
 'Halsted St & 59th St',
 'Racine Ave & 61st St',
 'Racine Ave & 65th St',
 'Vincennes Ave & 75th St',
 'Damen Ave & 59th St',
 'Ashland Ave & 66th St',
 'Rhodes Ave & 71st St',
 'Ashland Ave & 69th St',
 'Leavitt St & Belmont Ave',
 'Wentworth Ave & Cermak Rd',
 'Western Ave & Fillmore St',
 'Racine Ave & Washington Blvd',
 'Wood St & Chicago Ave',
 'Eggleston Ave & 69th St',
 'Stewart Ave & 63rd St',
 'South Chicago Ave & Elliot Ave',
 'Smith Park',
 'Leavitt St & Division St',
 'Halsted St & 78th St',
 'Damen Ave & Walnut (Lake) St',
 'Stewart Ave & 83rd St',
 'Ashland Ave & 78th St',
 'Ashland Ave & 73rd St',
 'Halsted St & 73rd St',
 'hubbard_test_lws',
 'WATSON TESTING - DIVVY',
 'State St & 95th St',
 'W Oakdale Ave & N Broadway',
 'Loomis Blvd & 84th St',
 'Prospect Sq & 91st St',
 'Indiana Ave & 103rd St',
 'Elizabeth St & 92nd St',
 'Marquette Ave & 89th St',
 'Baltimore Ave & 87th St',
 'Oglesby Ave & 100th St',
 'Eberhart Ave & 91st St',
 'Summit Ave & 86th St',
 'W Armitage Ave & N Sheffield Ave',
 'Houston Ave & 92nd St',
 'Dauphin Ave & 103rd St',
 'Halsted St & 96th St',
 'Cottage Grove Ave & 111th Pl',
 'East End Ave & 87th St',
 'Ashland Ave & 74th St',
 'Wentworth Ave & 104th St',
 'Burling St & Diversey Pkwy',
 'Constance Ave & 95th St',
 'State St & 123rd St',
 'Eberhart Ave & 131st St',
 'Big Marsh Park',
 'Stony Island Ave & 90th St',
 'Dauphin Ave & 87th St',
 'Halsted St & 111th St',
 'Major Taylor Trail & 115th St',
 'Greenwood Ave & 91st St',
 'Jeffery Blvd & 91st St',
 'Wood St & Taylor St (Temp)',
 'Halsted St & 104th St',
 'Throop St & Taylor St',
 'Greenwood Ave & 97th St',
 'Vernon Ave & 107th St',
 'Avenue O & 134th St',
 'Torrence Ave & 106th St',
 'Eggleston Ave & 92nd St',
 'Michigan Ave & 114th St',
 'Ewing Ave & Burnham Greenway',
 'Clyde Ave & 87th St',
 'Torrence Ave & 126th Pl',
 'Walden Pkwy & 100th St',
 'Burnham Greenway & 105th St',
 'Major Taylor Trail & 124th St',
 'Avenue O & 118th St',
 'N Clark St & W Elm St',
 'St. Louis Ave & Fullerton Ave',
 'New St & Illinois St',
 'Hale Ave & 107th St',
 'Bradley Park',
 'Commercial Ave & 130th St',
 'Vincennes Ave & 104th St',
 'Western Ave & 111th St',
 'Ada St & 113th St',
 'Lawndale Ave & 111th St',
 'Homewood Ave & 115th St',
 'Western Ave & 104th St',
 'Hegewisch Metra Station',
 'S Michigan Ave & E 118th St',
 'Dodge Ave & Main St',
 'Base - 2132 W Hubbard Warehouse',
 'N Green St & W Lake St']

In [7]:
len(allSt20)

694

In [8]:
removed20 = list(set(allSt20) - set(allSt21))
len(removed20)

24

In [9]:
added21 = list(set(allSt21) - set(allSt20))
added21.pop(0)

nan

In [10]:
len(added21)

177

# Plotting all added stations of 2020 and removed stations of 2019

In [11]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [12]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [13]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [14]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [15]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [16]:
#Adding co-ordinates to all the stations that are added this year
n21 = {k: locations[k] for k in locations.keys() & set(added21)}

#printing out the first item in the dictionary to check the key-value pair
list(n21.items())[0]

('Francisco Ave & Bloomingdale Ave',
 {'lat': 41.91395900000001, 'long': -87.699368})

In [17]:
#Converting the dictionary into dataframe
st21 = pd.DataFrame.from_dict(n21, orient='index')

In [18]:
#Resetting the index
st21.reset_index(inplace=True)

#Renaming the index column to Station name
st21 = st21.rename(columns = {'index':'Station Name'})

In [19]:
#Adding a column with values of color for every station
st21['color'] = st21.apply(lambda row: 'beige', axis=1)

#printing to check the structure of the dataframe
st21.head(1)

,Station Name,lat,long,color
0,Francisco Ave & Bloomingdale Ave,41.913959,-87.699368,beige


In [20]:
removed20 = ['MTL-ECO5.1-01',
 'HUBBARD ST BIKE CHECKING (LBS-WH-TEST)',
 'Throop (Loomis) St & Taylor St',
 'Smith Park (*)',
 'Ashland Ave & 69th St',
 'Chicago Ave & Dempster St',
 'HQ QR',
 'Wentworth Ave & Cermak Rd (Temp)',
 'hubbard_test_lws',
 'Stewart Ave & 63rd St (*)',
 'Ashland Ave & 73rd St',
 'Wood St & Chicago Ave (*)',
 'Damen Ave & Walnut (Lake) St (*)',
 'Leavitt St & Division St (*)',
 'Wood St & Taylor St',
 'Racine Ave & Washington Blvd (*)',
 'Western Ave & Fillmore St (*)',
 'Leavitt St & Belmont Ave (*)',
 'Burling St (Halsted) & Diversey Pkwy (Temp)',
 'Eggleston Ave & 69th St (*)',
 'Drake Ave & Fullerton Ave',
 'McClurg Ct & Illinois St',
 'Jeffery Blvd & 91st St',
 'WATSON TESTING - DIVVY']

In [21]:
#adding co-ordinates for the removed stations in 2020
rm20 = {k: locations[k] for k in locations.keys() & set(removed20)}

#printing out the first item in the dictionary to check the key-value pair
list(rm20.items())[0]

('McClurg Ct & Illinois St', {'lat': 41.89102000000001, 'long': -87.6173})

In [22]:
#Converting the dictionary into dataframe
rm20 = pd.DataFrame.from_dict(rm20, orient='index')

In [23]:
#Resetting the index
rm20.reset_index(inplace=True)

#Renaming the index column to Station name
rm20 = rm20.rename(columns = {'index':'Station Name'})

In [24]:
#Adding a column with values of color for every station
rm20['color'] = rm20.apply(lambda row: 'black', axis=1)

#printing to check the structure of the dataframe
rm20.head(1)

,Station Name,lat,long,color
0,McClurg Ct & Illinois St,41.89102,-87.6173,black


# Location Marker map for interactivity purpose (Station Name)

In [25]:
# Location Marker map for interactivity purpose (Station Name)#merging both the added21 and removed20 dataframes
merged = pd.concat([rm20, st21], axis =0)

In [26]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2021
## Saving the map for converting into gif

In [27]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2021')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "darkgreen",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st21.apply(plotDot, axis = 1)

m

In [28]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [29]:
m.save('Gif_circlemarker/2021.html')

In [30]:
with fileinput.FileInput('Gif_circlemarker/2021.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [31]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2021.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2021.png')
browser.quit()

In [32]:
image = Image.open('ss2021.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2021.png')

# Heatmap 2021
## Saving the map for converting into gif

In [33]:
numlist = st21.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2021')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [34]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [35]:
m.save('Gif_heatmap/2021.html')

In [36]:
with fileinput.FileInput('Gif_heatmap/2021.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [37]:
1# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2021.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2021.png')
browser.quit()

In [39]:
image = Image.open('ss2021.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2021.png')